<a href="https://colab.research.google.com/github/weso500/QMLIIOT/blob/main/TEPFault4DataPrep_Semi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyreadr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.3/418.3 kB 8.8 MB/s eta 0:00:00


In [ ]:
import pyreadr
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [ ]:
df_FaultFree = pyreadr.read_r(r'/content/drive/MyDrive/FDA Journal/TEP_FaultFree_Training.RData')['fault_free_training']
df_Faulty = pyreadr.read_r(r'/content/drive/MyDrive/FDA Journal/TEP_Faulty_Training.RData')['faulty_training']

In [ ]:
DF = pd.concat([df_FaultFree,df_Faulty]) # Join both fault free and faulty dataset
DF.head()

,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,0.0,1.0,1,0.25038,3674.0,4529.0,9.2320,26.889,42.402,2704.3,...,53.744,24.657,62.544,22.137,39.935,42.323,47.757,47.510,41.258,18.447
1,0.0,1.0,2,0.25109,3659.4,4556.6,9.4264,26.721,42.576,2705.0,...,53.414,24.588,59.259,22.084,40.176,38.554,43.692,47.427,41.359,17.194
2,0.0,1.0,3,0.25038,3660.3,4477.8,9.4426,26.875,42.070,2706.2,...,54.357,24.666,61.275,22.380,40.244,38.990,46.699,47.468,41.199,20.530
3,0.0,1.0,4,0.24977,3661.3,4512.1,9.4776,26.758,42.063,2707.2,...,53.946,24.725,59.856,22.277,40.257,38.072,47.541,47.658,41.643,18.089
4,0.0,1.0,5,0.29405,3679.0,4497.0,9.3381,26.889,42.650,2705.1,...,53.658,28.797,60.717,21.947,39.144,41.955,47.645,47.346,41.507,18.461


In [ ]:
columns_to_keep = ['faultNumber', 'simulationRun', 'sample', 'xmeas_7', 'xmeas_9', 'xmeas_21', 'xmv_10']
DF = DF[columns_to_keep]
display(DF.head())

,faultNumber,simulationRun,sample,xmeas_7,xmeas_9,xmeas_21,xmv_10
0,0.0,1.0,1,2704.3,120.41,94.640,41.258
1,0.0,1.0,2,2705.0,120.41,94.595,41.359
2,0.0,1.0,3,2706.2,120.42,94.605,41.199
3,0.0,1.0,4,2707.2,120.39,94.473,41.643
4,0.0,1.0,5,2705.1,120.39,94.678,41.507


In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
fault_free = DF[DF['faultNumber']==0].iloc[::50,3:]


In [ ]:
TEPtrain = fault_free.sample(n=500, random_state=42)
TEP_test = fault_free.drop(TEPtrain.index).sample(n=50, random_state=42)

display("TEPtrain head:", TEPtrain.head())
display("TEP_test head:", TEP_test.head())

'TEPtrain head:'

,xmeas_7,xmeas_9,xmeas_21,xmv_10
75050,2699.1,120.39,94.477,40.620
129300,2715.8,120.40,94.559,41.192
132650,2703.4,120.35,94.644,39.960
52750,2704.3,120.43,94.471,41.958
35250,2691.5,120.40,94.645,40.621


'TEP_test head:'

,xmeas_7,xmeas_9,xmeas_21,xmv_10
129800,2699.9,120.42,94.714,40.059
205550,2704.1,120.40,94.604,41.269
166350,2712.4,120.42,94.422,42.076
167300,2698.1,120.36,94.815,40.781
26700,2689.4,120.43,94.680,41.396


In [ ]:
faulty = DF[DF['faultNumber']==4].iloc[::30,3:]

In [ ]:
faulty_samples = faulty.sample(n=10, random_state=42, replace=False)
TEPtrain_with_faulty = pd.concat([TEPtrain, faulty_samples])
display("TEPtrain with faulty samples head:", TEPtrain_with_faulty.head())

'TEPtrain with faulty samples head:'

,xmeas_7,xmeas_9,xmeas_21,xmv_10
75050,2699.1,120.39,94.477,40.620
129300,2715.8,120.40,94.559,41.192
132650,2703.4,120.35,94.644,39.960
52750,2704.3,120.43,94.471,41.958
35250,2691.5,120.40,94.645,40.621


In [ ]:
TEPtrain = sc.fit_transform(TEPtrain_with_faulty)

In [ ]:
faulty_samples = faulty.sample(n=10, random_state=42)
TEPtest_with_faulty = pd.concat([pd.DataFrame(TEP_test, columns=faulty_samples.columns), faulty_samples])
display(TEPtest_with_faulty)

,xmeas_7,xmeas_9,xmeas_21,xmv_10
129800,2699.9,120.42,94.714,40.059
205550,2704.1,120.40,94.604,41.269
166350,2712.4,120.42,94.422,42.076
167300,2698.1,120.36,94.815,40.781
26700,2689.4,120.43,94.680,41.396
147700,2700.4,120.41,94.557,40.814
106700,2707.2,120.38,94.711,40.717
8700,2711.7,120.41,94.575,40.813
216900,2724.5,120.41,94.445,41.970
203450,2688.5,120.41,94.643,40.786


In [ ]:
TEPtest_final = sc.transform(TEPtest_with_faulty)

In [ ]:
pd.DataFrame(TEPtest_final).to_csv('TEP4Test_semi.csv', index=False)

In [ ]:
pd.DataFrame(TEPtrain).to_csv('TEP4Train_semi.csv', index=False)